In [1]:
from datagrunt.csvfile import CSVReader

### It's recommened to use the `duckdb` engine when querying CSV files. These query examples will work with the Polars engine exactly the same way as presented, but Datagrunt converts all data to strings when using DuckDB and it better preserves the integrity of the data due to not coercing into inferred datatypes. You'll see below that you can use SQL and you can optionally cast columns to different data types as needed.

### For example, you'll notice there are postal codes in the data below. Usually, postal codes are read as integers by most processing engines. Many postal codes begin with a `0`. If postal codes are read as an integer, the leading `0` will be dropped erroneously. This can be fixed by using the `lpad` function in Python, but we'd prefer to present the data in its original form as a string as opposed to having the data altered due to data type conversion. 

In [2]:
csv_file = 'data/electric_vehicle_population_data.csv'
engine = 'duckdb'

In [3]:
dg = CSVReader(csv_file, engine=engine) # set duckdb as the processing engine.

In [4]:
dg.get_sample() # return sample of the data to get a peek at the schema

┌────────────┬───────────┬──────────────┬───┬──────────────────────┬──────────────────────┬───────────────────┐
│ VIN (1-10) │  County   │     City     │ … │   Vehicle Location   │   Electric Utility   │ 2020 Census Tract │
│  varchar   │  varchar  │   varchar    │   │       varchar        │       varchar        │      varchar      │
├────────────┼───────────┼──────────────┼───┼──────────────────────┼──────────────────────┼───────────────────┤
│ 5YJSA1E28K │ Snohomish │ Mukilteo     │ … │ POINT (-122.29943 …  │ PUGET SOUND ENERGY…  │ 53061042001       │
│ 1C4JJXP68P │ Yakima    │ Yakima       │ … │ POINT (-120.468875…  │ PACIFICORP           │ 53077001601       │
│ WBY8P6C05L │ Kitsap    │ Kingston     │ … │ POINT (-122.517835…  │ PUGET SOUND ENERGY…  │ 53035090102       │
│ JTDKARFP1J │ Kitsap    │ Port Orchard │ … │ POINT (-122.653005…  │ PUGET SOUND ENERGY…  │ 53035092802       │
│ 5UXTA6C09N │ Snohomish │ Everett      │ … │ POINT (-122.203234…  │ PUGET SOUND ENERGY…  │ 53061041605 

In [5]:
df = dg.to_dataframe()
df

VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""5YJSA1E28K""","""Snohomish""","""Mukilteo""","""WA""","""98275""","""2019""","""TESLA""","""MODEL S""","""Battery Electric Vehicle (BEV)""","""Clean Alternative Fuel Vehicle…","""270""","""0""","""21""","""236424583""","""POINT (-122.29943 47.912654)""","""PUGET SOUND ENERGY INC""","""53061042001"""
"""1C4JJXP68P""","""Yakima""","""Yakima""","""WA""","""98901""","""2023""","""JEEP""","""WRANGLER""","""Plug-in Hybrid Electric Vehicl…","""Not eligible due to low batter…","""21""","""0""","""15""","""249905295""","""POINT (-120.4688751 46.6046178…","""PACIFICORP""","""53077001601"""
"""WBY8P6C05L""","""Kitsap""","""Kingston""","""WA""","""98346""","""2020""","""BMW""","""I3""","""Battery Electric Vehicle (BEV)""","""Clean Alternative Fuel Vehicle…","""153""","""0""","""23""","""260917289""","""POINT (-122.5178351 47.7981436…","""PUGET SOUND ENERGY INC""","""53035090102"""
"""JTDKARFP1J""","""Kitsap""","""Port Orchard""","""WA""","""98367""","""2018""","""TOYOTA""","""PRIUS PRIME""","""Plug-in Hybrid Electric Vehicl…","""Not eligible due to low batter…","""25""","""0""","""26""","""186410087""","""POINT (-122.6530052 47.4739066…","""PUGET SOUND ENERGY INC""","""53035092802"""
"""5UXTA6C09N""","""Snohomish""","""Everett""","""WA""","""98208""","""2022""","""BMW""","""X5""","""Plug-in Hybrid Electric Vehicl…","""Clean Alternative Fuel Vehicle…","""30""","""0""","""44""","""186076915""","""POINT (-122.2032349 47.8956271…","""PUGET SOUND ENERGY INC""","""53061041605"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""JTMAB3FVXR""","""Snohomish""","""Snohomish""","""WA""","""98290""","""2024""","""TOYOTA""","""RAV4 PRIME""","""Plug-in Hybrid Electric Vehicl…","""Clean Alternative Fuel Vehicle…","""42""","""0""","""44""","""262809249""","""POINT (-122.0483457 47.9435765…","""PUGET SOUND ENERGY INC""","""53061052402"""
"""7FCTGAAA7P""","""Pierce""","""Orting""","""WA""","""98360""","""2023""","""RIVIAN""","""R1T""","""Battery Electric Vehicle (BEV)""","""Eligibility unknown as battery…","""0""","""0""","""2""","""252195450""","""POINT (-122.197791 47.0948565)""","""PUGET SOUND ENERGY INC||CITY O…","""53053070100"""
"""1V2GNPE87P""","""Spokane""","""Spokane""","""WA""","""99201""","""2023""","""VOLKSWAGEN""","""ID.4""","""Battery Electric Vehicle (BEV)""","""Eligibility unknown as battery…","""0""","""0""","""3""","""227314790""","""POINT (-117.428902 47.658268)""","""MODERN ELECTRIC WATER COMPANY""","""53063002300"""


### Write a SQL query and use a f string to interpolate the database table name provided automatically by Datagrunt powered by DuckDB.

In [6]:
query = f"""
WITH core AS (
    SELECT
        City AS city,
        "VIN (1-10)" AS vin
    FROM {dg.db_table}
)
SELECT
    city,
    COUNT(vin) AS vehicle_count
FROM core
GROUP BY 1
ORDER BY 2 DESC
"""

In [7]:
dg.query_data(query)

┌───────────────┬───────────────┐
│     city      │ vehicle_count │
│    varchar    │     int64     │
├───────────────┼───────────────┤
│ Seattle       │         32602 │
│ Bellevue      │          9960 │
│ Redmond       │          7165 │
│ Vancouver     │          7081 │
│ Bothell       │          6602 │
│ Kirkland      │          5883 │
│ Renton        │          5835 │
│ Sammamish     │          5795 │
│ Olympia       │          4830 │
│ Tacoma        │          4204 │
│   ·           │             · │
│   ·           │             · │
│   ·           │             · │
│ Minneapolis   │             1 │
│ Hamilton      │             1 │
│ Bedford       │             1 │
│ Waldorf       │             1 │
│ Las Cruces    │             1 │
│ Conyers       │             1 │
│ Benicia       │             1 │
│ Ellicott City │             1 │
│ Sausalito     │             1 │
│ Fallon        │             1 │
├───────────────┴───────────────┤
│      764 rows (20 shown)      │
└─────────────

### Run the query again and return to a Polars dataframe

In [8]:
dg.query_data(query).pl()

city,vehicle_count
str,i64
"""Seattle""",32602
"""Bellevue""",9960
"""Redmond""",7165
"""Vancouver""",7081
"""Bothell""",6602
…,…
"""Gunpowder""",1
"""Holden Village""",1
"""Yorktown""",1


### Run the query yet again and return a Pandas dataframe

In [9]:
dg.query_data(query).to_df()

,city,vehicle_count
0,Seattle,32602
1,Bellevue,9960
2,Redmond,7165
3,Vancouver,7081
4,Bothell,6602
...,...,...
759,Deer Harbor,1
760,North Las Vegas,1
761,Vista,1
762,Tempe,1


### Select directly from the dataframe object above assigned to the `df` variable. Do not use a f string to interpolate the `df` variable as an object. Instead, reference the `df` object in the `FROM` statement that's embedded in the query string.

### Also notice that we're using SQL to make the `VIN (1-10)` column name more friendly with a SQL alias in the CTE.

In [10]:
df_query = f"""
WITH core AS (
    SELECT
        City AS city,
        "VIN (1-10)" AS vin
    FROM df
)
SELECT
    city,
    COUNT(vin) AS vehicle_count
FROM core
GROUP BY 1
ORDER BY 2 DESC
"""

### Return the results from querying the dataframe.

In [11]:
dg.query_data(df_query)

┌──────────────────┬───────────────┐
│       city       │ vehicle_count │
│     varchar      │     int64     │
├──────────────────┼───────────────┤
│ Seattle          │         32602 │
│ Bellevue         │          9960 │
│ Redmond          │          7165 │
│ Vancouver        │          7081 │
│ Bothell          │          6602 │
│ Kirkland         │          5883 │
│ Renton           │          5835 │
│ Sammamish        │          5795 │
│ Olympia          │          4830 │
│ Tacoma           │          4204 │
│   ·              │             · │
│   ·              │             · │
│   ·              │             · │
│ Lamont           │             1 │
│ Dickinson        │             1 │
│ Saratoga Springs │             1 │
│ Gunpowder        │             1 │
│ Holden Village   │             1 │
│ Yorktown         │             1 │
│ Ridgecrest       │             1 │
│ Startup          │             1 │
│ Sacramento       │             1 │
│ Washtucna        │             1 │
├

### Query the dataframe using the `df_query` SQL query string and return the results to a Polars dataframe. The benefit here is you can use the power of SQL to transform data in a dataframe and return the results to a new dataframe object.

In [12]:
dataframe_query_polars_df = dg.query_data(df_query).pl()
dataframe_query_polars_df

city,vehicle_count
str,i64
"""Seattle""",32602
"""Bellevue""",9960
"""Redmond""",7165
"""Vancouver""",7081
"""Bothell""",6602
…,…
"""Vista""",1
"""Tempe""",1
"""Green Bay""",1


### You can optionally use the `to_df` method to return the dataframe results as a Pandas dataframe. The beauty of this is that we're querying from a Polars dataframe, we use SQL to transform the data powered by DuckDB, and then we're returning the results to a Pandas dataframe. Datagrunt is designed to offer power, simplicity, and flexibility when working with CSV files.

In [13]:
dataframe_query_pandas_df = dg.query_data(df_query).to_df()
dataframe_query_pandas_df

,city,vehicle_count
0,Seattle,32602
1,Bellevue,9960
2,Redmond,7165
3,Vancouver,7081
4,Bothell,6602
...,...,...
759,Vista,1
760,Tempe,1
761,Green Bay,1
762,Waverly,1
